# DSO560HW3

### Xiangyu Huang (xhuang16)

## TF-IDF

You are an analyst working for Amazon's product team, and charged with identifying areas for improvement for the toy reviews.

Using the **amazon-[poor|good]-reviews.csv** dataset, clean and parse the text reviews. Explain the decisions you make:
- why remove/keep stopwords?
- which stopwords to remove?
- stemming versus lemmatization?
- regex cleaning and substitution?
- adding in custom stopwords?
- what `n` for your `n-grams`?

Finally, generate a TF-IDF report that explains for a business (non-technical) stakeholder:
* the features your analysis showed that customers cited as reasons for a poor review
* the features your analysis showed that customers cited as reasons for a good review
* the most common issues identified from your analysis that generated customer dissatisfaction.

Explain to what degree the TF-IDF findings make sense - what are its limitations?

In [2]:
import pandas as pd
import math
import re
import matplotlib.pyplot as plt 
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from spacy.matcher import Matcher 
from spacy.tokens import Span
from scipy.spatial.distance import cosine
from itertools import product
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

In [11]:
nltk.download('punkt') 
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/huangsky/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/huangsky/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/huangsky/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [1]:
poor = open("poor_amazon_toy_reviews.txt", "r")
good = open("good_amazon_toy_reviews.txt", "r")
good_df = pd.DataFrame(open("good_amazon_toy_reviews.txt", "r"), columns=["comments"])
poor_df = pd.DataFrame(open("poor_amazon_toy_reviews.txt", "r"), columns = ["comments"])

NameError: name 'pd' is not defined

In [ ]:
good_df['comments'] = good_df['comments'].str.lower()
poor_df['comments'] = poor_df['comments'].str.lower()

 - In the following cells, I will change some of the stop words.
 - I add some stop words including  "i/he/she'll" and some other meangingless words that are not included in the nltk stopword list but may be common in the speech. I also exclude words include symbols in them. 
 - On the other side, I keep the words with negative meaning such as not, nor and anything end up with n't or n. 

In [ ]:
stop_words = nltk.corpus.stopwords.words('english')
remove_stop_words = ["not", "nor"] 
remove_stop_words.extend(re.findall(r"\b(\w+n't|\w{3,}n)\b", sentence))
remove_stop_words
for each in remove_stop_words:
    stop_words.remove(each)
my_stopwords = ["so", "i'm", "i've" ,"i'll","really","that's","he'll", "she'll", "when", "been", "we'll", "they'll"
               "would", "/><br" , "<br",'&#34;' ,'"', 'even' , 'could' , 'may'] 
stop_words.extend(my_stopwords)
stop_words

- I will use the function in the cell below to clean up all the comments.
- **I didn't use nltk tokenize because it will divide "don't" into two words "do and n't". which is not what I am looking for. In this function, I will jutst replace the symbol combination with empty space while for the other symbol might be the end of sentence, I replace them with a white space. Then, I will exclude the stop words and output the cleaned result.**

In [ ]:
def clean_sentence( raw_line ):
    line = re.sub('&#34;', '' ,raw_line, flags = re.IGNORECASE)
    line = re.sub(r'[.!,]', ' ',line, flags = re.IGNORECASE)
    result = [word.lower() for word in line.split() if word.lower() not in stop_words]
    return " ".join(result)

In [731]:
good_df['line'] = good_df.apply(lambda x: clean_sentence(x['comments']), axis = 1)
poor_df['line'] = poor_df.apply(lambda x: clean_sentence(x['comments']), axis = 1)

In [732]:
good_df['line'] = good_df['line'].str.replace(r"\b(?:year|yr)s?(?:-|\s)?(?:old)s?\b", "year_old_age")
poor_df['line'] = poor_df['line'].str.replace(r"\b(?:year|yr)s?(?:-|\s)?(?:old)s?\b", "year_old_age")

## I apply regex to replace the combination of "year/yr(s) old(s) into year_old_age."

In [733]:
def count_words(lines, delimiter=" "):  ## Function from class
    
    words = Counter() 
    for line in lines:
        for word in line.split(delimiter):
            words[word] += 1 
    return words
good_words_count = count_words(good_df['line'])


good_words_count['year_old_age']
## The regex replaces 8295 observations in the datasets and makes it easier for analysis"

8295

 - **For the positive and negative review, I choose to apply lemmatization rather than stemming here. When we check the positive and negative reivew, we want to know exactly which tokens or things are related to the comment results. In other words, I think the precision is more important than recall rate in this case.**
 - The following cells is from: https://gaurav5430.medium.com/using-nltk-for-lemmatizing-sentences-c1bfff963258 for lemmatiziation in context.

In [12]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)


In [735]:
good_df['line_lemmatization'] = good_df['line'].apply(lambda x: lemmatize_sentence(x))
poor_df['line_lemmatization'] = poor_df['line'].apply(lambda x: lemmatize_sentence(x))
## Lemmatize each comments with the function.

In [736]:
vectorizer = TfidfVectorizer(ngram_range=(2,3),
                             max_features=5000, min_df = 0.01 )
corpus = list(good_df["line_lemmatization"].values)
X = vectorizer.fit_transform(corpus)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)

## Due to my laptop's hardcore limitation. I choose max_features of 5000 and min_df 0.01 to exlcude the lowest 1% words in 2 grams.


In [737]:
tf_idf = tf_idf.sum(axis=1)
score = pd.DataFrame(tf_idf, columns=["score"])
score.sort_values(by="score", ascending=False, inplace=True)

In [738]:
score

,score
daughter love,2907.241168
son love,2700.775878
kid love,2665.126097
grandson love,2063.942678
well make,1862.516970
great product,1628.233919
good quality,1524.802087
granddaughter love,1463.654327
work great,1370.081800
year_old_age love,1352.748115


In [739]:
vectorizer = TfidfVectorizer(ngram_range=(2,3),
                             max_features = 5000, min_df = 0.01 )
corpus = list(poor_df["line_lemmatization"].values)
X = vectorizer.fit_transform(corpus)
terms = vectorizer.get_feature_names()
tf_idf_poor = pd.DataFrame(X.toarray().transpose(), index=terms)
tf_idf_poor = tf_idf_poor.sum(axis=1)
score_poor = pd.DataFrame(tf_idf_poor, columns=["score"])
score_poor.sort_values(by="score", ascending=False, inplace=True)
score_poor

,score
waste money,528.530071
do work,401.930329
not work,362.008946
not worth,354.915834
not buy,350.421040
would not,311.960797
look like,284.336023
poor quality,231.587464
cheaply make,191.278861
do buy,175.627111


- TF-IDF score is calculated by term frequency and inverse document frequency. The higher scores means that these words are more relevant to the whole text documents. For TF-IDF, a high score usually means the word rarely appears in common speech but achieve high relevancy in the context. 

- According to the score above, we can see that kids/daughter/son love have the highest TF-IDF scores. Also, the phrases good quality/ great price are another two keywords achieve high TF-IDF scores. Thus, we can conclude the toy quality and price are attractive for consumers. Also, purchases made for daughters are more relevant than these for sons. Thus, girls at young age tend to make their parents to give positive feedback. 

- On the other side, the phrases with high score TF-IDF for poor review are ‘waste money, not work, not worth, poor quality’. From the scores above, we can conclude that customers are mostly dissatisfied with these features: ‘cheaply make’, ‘fell apart’, and ‘product quality’. 

- TF-IDF could be very useful for the searching engine to find related information. However, the limitation of TF-IDF is that it doesn’t consider the position and context of the words. Thus, it can only give the relevancy but fail to give a reasonable meaning for the words in-context meaning. 


## Similarity and Word Embeddings

Using
* `CountVectorizer`
* `TfIdfVectorizer`

Identify the most similar reviews from the McDonalds Yelp dataset.

Then, using the `spacy`, identify the top 10 pairs of most similar words used in the reviews.

In [14]:
mcd_review = pd.read_csv("mcdonalds-yelp-negative-reviews.csv", encoding = 'latin-1' )
mcd_review['review'] = mcd_review['review'].str.lower()

mcd_review['review'] = mcd_review['review'].str.replace(r"\b(\w+)?burgers?\b|\b(\w+)?sandwich(\w+)?\b|\b(?:ham|chesse|steak|chicken|double|triple)s?(?:\s)(?:burger|bun|sandwich)s?\b" ,
                                                        'burger')

mcd_review['review'] = mcd_review['review'].str.replace(r"\bmcd\b|\bmcdonald's\b|\bmacd\b|\bmc\b|\b(\w+)?mcdonald?(\w+)?\b" , 
                                                        'macdonald')

stemmer = nltk.stem.porter.PorterStemmer()
lemmatizer = WordNetLemmatizer()
## I define a function for lines using method either lemma or stemming
def stemm_lemm(lines, method):
    result= []
    for line in lines:
        words = line.split()
        new_words = []
        for word in words:
            word = method(word)
            new_words.append(word)
        results = " ".join(new_words)
        result.append(results)
    return result
mcd_review['stemmed_review'] = stemm_lemm(mcd_review['review'], stemmer.stem)
mcd_review['stemmed_review']

0       i'm not a huge mcd lover, but i'v been to bett...
1       terribl custom service. i came in at 9:30pm an...
2       first they "lost" my order, actual they gave i...
3       i see i'm not the onli one give 1 star. onli b...
4       well, it' macdonald, so you know what the food...
                              ...                        
1520    i enjoy the part where i repeatedli ask if i h...
1521    worst macdonald i'v been in in a long time! di...
1522    when i am realli crave for macdonald, thi seem...
1523    two point right out of the gate: 1. thuggeri k...
1524    i want to grab breakfast one morn befor work s...
Name: stemmed_review, Length: 1525, dtype: object

In [15]:
vectorizer = CountVectorizer(ngram_range=(1, 1),
                             stop_words = 'english', binary = True , 
                             min_df = 2, max_features = 2000, token_pattern = r'[a-z]{3,}')


In [16]:
X= vectorizer.fit_transform(mcd_review['stemmed_review'])

In [17]:
vectorizer_df = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names())
vectorizer_df

,abl,abov,absolut,absurd,accept,access,accommod,accord,account,accuracy,...,yesterday,yikes,yogurt,york,young,younger,yuck,yum,yummy,zero
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1521,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1522,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1523,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


 - The following code cells, I will use function cosine_similarity and our vectorizer_df to find the most similar reviews.
 - First, I will iterate through the cosine_similarity(vectorizer_df) with each documents. This will end up with an array with cosine distance to each of the other sentence.
 - Then , I will exclude values greater than 0.99 to exclude the same sentences by setting them to 0. I use 0.99 here because sometimes the same sentence may result as 0.99999.
 - After set the same sentence distance to 0, I will find the max consine similarity result for each sentence.
 - Then, when I have the maximum 10 results, I will find the index for these lines and output the sentences with their corresponding line together. 
 
 
 - I should put these for loops in one, but it takes too long to run. 

In [18]:
max_similarity = []

for i in range(len(vectorizer_df)):
    result = cosine_similarity(vectorizer_df)[i]
    result[result>= 0.99] = 0
    max_similarity.append(np.max(result))
max_similarity

[0.38729833462074176,
 0.29277002188455997,
 0.2637521893583148,
 0.3380617018914066,
 0.34299717028501764,
 0.3370999312316211,
 0.2959181771496432,
 0.24333213169614382,
 0.35791699479543954,
 0.3377830118332837,
 0.32071349029490925,
 0.26111648393354675,
 0.30618621784789724,
 0.36181361349331637,
 0.36514837167011066,
 0.4082482904638631,
 0.3418817293789138,
 0.3464101615137754,
 0.28057938012106604,
 0.5222329678670936,
 0.20100756305184242,
 0.36293309315564887,
 0.297456538265446,
 0.3198010745334156,
 0.32128773156099955,
 0.2279211529192759,
 0.3235751144647171,
 0.31234752377721214,
 0.3872983346207416,
 0.28603877677367767,
 0.27695585470349865,
 0.33968311024337877,
 0.37267799624996495,
 0.30822603282490524,
 0.311187957132071,
 0.372677996249965,
 0.4082482904638631,
 0.2667325346846322,
 0.42857142857142866,
 0.2521693354133353,
 0.26490647141300877,
 0.408248290463863,
 0.32274861218395134,
 0.3446278114803676,
 0.3113995776646092,
 0.42857142857142866,
 0.37499999999

In [23]:
theset = frozenset(max_similarity)
theset = sorted(theset, reverse=True)

In [24]:
theset

[0.9214064786997642,
 0.816496580927726,
 0.7453559924999298,
 0.6666666666666669,
 0.6123724356957946,
 0.5773502691896258,
 0.5715476066494082,
 0.5714285714285713,
 0.545544725589981,
 0.5345224838248488,
 0.5303300858899106,
 0.5222329678670936,
 0.5039526306789696,
 0.502518907629606,
 0.5,
 0.49999999999999994,
 0.4999999999999999,
 0.4961389383568338,
 0.4834937784152281,
 0.47809144373375745,
 0.4743416490252569,
 0.47434164902525683,
 0.4714045207910318,
 0.4714045207910316,
 0.4629100498862758,
 0.4629100498862757,
 0.46188021535170054,
 0.4583492485141056,
 0.45643546458763856,
 0.4564354645876385,
 0.45643546458763845,
 0.45643546458763834,
 0.4558423058385518,
 0.45291081365783836,
 0.45226701686664544,
 0.4499770425732575,
 0.4472135954999579,
 0.44721359549995787,
 0.44543540318737396,
 0.4444444444444444,
 0.443484296991812,
 0.44194173824159216,
 0.44095855184409843,
 0.43852900965351466,
 0.43643578047198484,
 0.4364357804719848,
 0.4351941398892446,
 0.43386091563731

In [22]:
max_similarity.sort_values(ascending = False)

AttributeError: 'list' object has no attribute 'sort_values'

In [745]:
simi_array = np.array(max_similarity)
simi_array

array([0.38729833, 0.29277002, 0.26375219, ..., 0.37532595, 0.26207121,
       0.34585723])

In [746]:
index_simi = np.argpartition(simi_array, -10)[-10:]
index_simi   
## I use a code from stackoverflow to get the index for the 10 largest cosine similarity result. 
## https://stackoverflow.com/questions/6910641/how-do-i-get-indices-of-n-maximum-values-in-a-numpy-array
## I will use index to match up at last. 

array([1482,   76, 1014,  504,  841,  653,  139,  140,  911,  676])

In [747]:
for each in index_simi:
    result = cosine_similarity(vectorizer_df)[each]
    result[result>= 0.99] = 0
    index_pair = np.argpartition(result, -1)[-1:][0]
    print(mcd_review['stemmed_review'][each], "\n", 
          mcd_review['stemmed_review'][index_pair],
         "\n")

they are open. good service. 
 it good. 24hr 

best macdonald ice cream is here! 
 i love macdonald had 5 ice cream cone once. no regret 

i love macdonald had 5 ice cream cone once. no regret 
 i love the ice cream here! that the onli thing i get when i go to macdonald. 

i love the ice cream here! that the onli thing i get when i go to macdonald. 
 i love macdonald had 5 ice cream cone once. no regret 

they forgot my order:( i wa wait 15 min for my food. 
 forgot half my order, then made me wait 10 min to correct it. not veri fast food. 

forgot half my order, then made me wait 10 min to correct it. not veri fast food. 
 they forgot my order:( i wa wait 15 min for my food. 

worst macdonald ever!!!!!!!!!! they are unabl to get an order correct, went through the drive through last night got home and no fri in my bag. got to the window and the woman run the window wa talk about her person life with a co worker laugh it up with her back t us make us wait instead of give us our ice coff

 - **The result above actually looks good to me. It only takes the most ismilar 5 pairs. Since the top ten are matching to each other here.**




- The following cells apply the spacy tokenst to pick the most similar words.
- We can see these words are similar type of words but they don't have similar meaning in context.

In [748]:
top_2000_words = vectorizer.get_feature_names()

In [749]:
tokens = nlp(f'{" ".join(top_2000_words)}')
similarity_tuples = []
for token1, token2 in product(tokens, repeat=2):
    similarity_tuples.append((token1, token2, token1.similarity(token2)))
similarities = pd.DataFrame(similarity_tuples, columns=["word1","word2", "score"])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  after removing the cwd from sys.path.


In [750]:
similarities

,word1,word2,score
0,abl,abl,1.000000
1,abl,abov,0.442692
2,abl,absolut,0.273940
3,abl,absurd,0.221417
4,abl,accept,0.138327
...,...,...,...
4032059,zero,younger,0.035352
4032060,zero,yuck,0.048143
4032061,zero,yum,0.285949
4032062,zero,yummy,0.359283


In [752]:
similarities[similarities["score"] < 1].sort_values(
    by="score", ascending=False).drop_duplicates(
    subset="score", keep="first").head(10)

,word1,word2,score
1982046,lowest,biggest,0.866666
2807834,reflect,follow,0.857942
2809843,refund,food,0.845026
2938116,run,cut,0.844103
2865074,requested,stated,0.842222
2744255,rare,professional,0.841434
680111,connect,regret,0.840450
2691219,push,doubl,0.832390
301502,biggest,closest,0.831569
2826225,regret,lock,0.829830


In [ ]:
nlp = spacy.load('en_core_web_sm', parser=False, entity=False)        
df['Tokens'] = df.Sumcription.apply(lambda x: nlp(x))    
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS        
spacy_stopwords.add('attach')
df['Lema_Token']  = df.Tokens.apply(lambda x: " ".join([token.lemma_ for token in x if token not in spacy_stopwords]))